## Setup

Load libraries

In [2]:
import os
import pickle
from snowmodels.swe import HillSWE
from snowmodels.density import SturmDensity, JonasDensity, PistochiDensity 

## Load Dataset

In [3]:
with open('../data/data_splits.pkl', 'rb') as f:
    data_splits = pickle.load(f)

## Statistical Models

### Density Prediction (Test Set)

In [4]:
density_results=(
    data_splits['X_test']
    .assign(
        Observed_Density=data_splits['y_test'],
        Jonas_Density=lambda x: x.loc[:, ('Snow_Depth', 'Date', 'Elevation')].apply(lambda y: JonasDensity().compute_density(snow_depth=y.Snow_Depth*0.01, month=y.Date.month_name(), elevation= y.Elevation), axis=1),
        Pistochi_Density=lambda x: x.loc[:, 'Pistochi_DOWY'].apply(lambda y: PistochiDensity().compute_density(DOY=y)),
        Sturm_Density=lambda x: x.loc[:, ('Snow_Depth', 'Sturm_DOWY', 'Snow_Class')].apply(lambda y: SturmDensity().compute_density(snow_depth=y.Snow_Depth, DOY=y.Sturm_DOWY, snow_class= y.Snow_Class), axis=1) 
    )
    .filter(items=['Snow_Class', 'Observed_Density', 'Jonas_Density', 'Pistochi_Density', 'Sturm_Density'])
    .reset_index(drop=True)
)

density_results.head()

,Snow_Class,Observed_Density,Jonas_Density,Pistochi_Density,Sturm_Density
0,Alpine,0.190909,0.229395,0.315,0.249035
1,Alpine,0.319048,0.285868,0.397,0.333826
2,Alpine,0.209524,0.258474,0.318,0.262308
3,Alpine,0.450649,0.348602,0.435,0.405091
4,Alpine,0.273913,0.236378,0.331,0.260724


### SWE Prediction (Test Set)

In [5]:
swe_results=(
    density_results
    .assign(
        Observed_SWE=data_splits['X_test'].reset_index().SWE,
        Snow_Depth=data_splits['X_test'].reset_index().Snow_Depth,
        Jonas_SWE=lambda x: x.Jonas_Density * x.Snow_Depth,
        Sturm_SWE=lambda x: x.Sturm_Density * x.Snow_Depth,
        Pistochi_SWE=lambda x: x.Pistochi_Density * x.Snow_Depth,
        Hill_SWE=data_splits['X_test'].reset_index()[['PPTWT', 'Temp_Diff', 'DOWY', 'Snow_Depth']].apply(lambda x: HillSWE().compute_swe(pptwt=x.PPTWT, TD=x.Temp_Diff, DOY=x.DOWY, snow_depth=x.Snow_Depth*10), axis=1)
    )
    .drop(
        columns=[
            'Observed_Density', 
            'Jonas_Density', 
            'Pistochi_Density', 
            'Sturm_Density', 
            'Snow_Depth',
        ]
    )
)

swe_results.head()

,Snow_Class,Observed_SWE,Jonas_SWE,Sturm_SWE,Pistochi_SWE,Hill_SWE
0,Alpine,16.002,19.227922,20.874140,26.40330,26.611937
1,Alpine,17.018,15.248220,17.806287,21.17598,17.396827
2,Alpine,22.352,27.573964,27.982968,33.92424,32.438612
3,Alpine,88.138,68.179618,79.227638,85.07730,92.874028
4,Alpine,16.002,13.809226,15.231518,19.33702,NaN


## Store Predictions to File

In [6]:
os.makedirs('../results', exist_ok=True)
os.makedirs('../results/density', exist_ok=True)
os.makedirs('../results/swe', exist_ok=True)

density_results.to_parquet('../results/density/statistical_models_predictions.parquet', index=False, compression='gzip')
swe_results.to_parquet('../results/swe/statistical_models_predictions.parquet', index=False, compression='gzip')